## Visualizing Experimental Results

In [1]:
import pandas as pd
import os

In [2]:
path = os.path.join(os.getcwd(), "artifacts", "results.csv")
df = pd.read_csv(path, sep=",")
df.head()

,Piece,Avg Time Delay,Right Note Ratio,Played Notes,Matched Notes,Score Notes
0,brahms_carlos,0.618422,0.201247,1123,226,2077
1,chopin_op60,1.212495,0.260978,4395,1147,4378
2,brahms_carlos,0.650345,0.192032,1255,241,2077
3,chopin_op09_No2,2.244669,0.283876,1321,375,1246
4,chopin_op15_No2,2.525406,0.549708,342,188,1136


## Exploring MIDI Degradation

In [35]:
import pandas as pd
import numpy as np
import partitura as pt
from numpy.lib.recfunctions import unstructured_to_structured
from mdtk.degradations import (
    pitch_shift,
    time_shift,
    onset_shift,
    offset_shift,
    remove_note,
    add_note,
)

In [36]:
performance = pt.load_performance("/home/manos/Desktop/JKU/data/midi/inventions/BachJS-BWV772.mid")
note_array = performance.note_array()
ppq = performance[0].ppq
mpq = performance[0].mpq
odf = pd.DataFrame(note_array)
odf.head()

,onset_sec,duration_sec,onset_tick,duration_tick,pitch,velocity,track,channel,id
0,0.125,0.125,2,2,60,64,0,1,P00_n0
1,0.250,0.125,4,2,62,64,0,1,P00_n1
2,0.375,0.125,6,2,64,64,0,1,P00_n2
3,0.500,0.125,8,2,65,64,0,1,P00_n3
4,0.625,0.125,10,2,62,64,0,1,P00_n4


In [37]:
df = pd.DataFrame(
    {
        "onset": note_array["onset_tick"],
        "pitch": note_array["pitch"],
        "dur": note_array["duration_tick"],
        "velocity": note_array["velocity"],
        "track": note_array["track"]
    }
)
df.head()


min_pitch = 36  # cant shift lower than 60
max_pitch = 110
distribution = [1, 0, 1, 0, 1, 0, 1]  # only shift min 2nd or maj 3rd up/down

In [43]:
deg_excerpt_1 = pitch_shift(
    df,
    min_pitch=min_pitch,
    max_pitch=max_pitch,
    distribution=distribution,
    seed=42
)
deg_excerpt_1["id"] = odf["id"]
deg_excerpt_1["channel"] = odf["channel"]
deg_excerpt_1["onset_sec"] = pt.utils.music.midi_ticks_to_seconds(deg_excerpt_1["onset"].to_numpy(), mpq=mpq, ppq=ppq)
deg_excerpt_1["duration_sec"] = pt.utils.music.midi_ticks_to_seconds(deg_excerpt_1["dur"].to_numpy(), mpq=mpq, ppq=ppq)
deg_excerpt_1.head()

,onset,track,pitch,dur,velocity,id,channel,onset_sec,duration_sec
0,2,0,60,2,64,P00_n0,1,0.125,0.125
1,4,0,62,2,64,P00_n1,1,0.250,0.125
2,6,0,64,2,64,P00_n2,1,0.375,0.125
3,8,0,65,2,64,P00_n3,1,0.500,0.125
4,10,0,62,2,64,P00_n4,1,0.625,0.125


array([( 2, 0, 60, 2, 64, 'P00_n0', 1), ( 4, 0, 62, 2, 64, 'P00_n1', 1),
       ( 6, 0, 64, 2, 64, 'P00_n2', 1), ( 8, 0, 65, 2, 64, 'P00_n3', 1),
       (10, 0, 62, 2, 64, 'P00_n4', 1), (12, 0, 64, 2, 64, 'P00_n5', 1),
       (14, 0, 60, 2, 64, 'P00_n6', 1), (16, 0, 67, 4, 64, 'P00_n7', 1),
       (18, 1, 48, 2, 64, 'P01_n0', 1), (20, 0, 72, 4, 64, 'P01_n1', 1)],
      dtype=[('onset', '<i8'), ('track', '<i8'), ('pitch', '<i8'), ('dur', '<i8'), ('velocity', '<i8'), ('id', 'O'), ('channel', '<i4')])

In [44]:
records = deg_excerpt_1.to_records(index=False)
new_note_array = np.array(records, dtype = records.dtype.descr)
ppart = pt.performance.PerformedPart.from_note_array(new_note_array, )
pt.save_performance_midi(ppart, "./artifacts/test.mid")